In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [71]:
train = pd.read_pickle("data/train_over_shrink.pickle")
test = pd.read_pickle("data/test.pickle")

In [72]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgbm

In [73]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

In [74]:
rfc = RandomForestClassifier(n_jobs=-1)
xgb_clf = xgb.XGBClassifier(n_jobs=-1, use_label_encoder=False)
lgbm_clf = lgbm.LGBMClassifier()
def model_fit(train):
    rfc.fit(train.iloc[:,:-1].to_numpy(), train["Class"].to_numpy())
    xgb_clf.fit(train.iloc[:,:-1].to_numpy(), train["Class"].to_numpy())
    lgbm_clf.fit(train.iloc[:,:-1].to_numpy(), train["Class"].to_numpy())

In [75]:
model_fit(train)

[22:18:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [76]:
train_prob = dict()
test_prob = dict()

train_prob["rfc"] = rfc.predict_proba(train.iloc[:,:-1]).T[1]
train_prob["xgb"] = xgb_clf.predict_proba(train.iloc[:,:-1]).T[1]
train_prob["lgbm"] = lgbm_clf.predict_proba(train.iloc[:,:-1]).T[1]

test_prob["rfc"] = rfc.predict_proba(test.iloc[:,:-1]).T[1]
test_prob["xgb"] = xgb_clf.predict_proba(test.iloc[:,:-1]).T[1]
test_prob["lgbm"] = lgbm_clf.predict_proba(test.iloc[:,:-1]).T[1]

/home/nabeel/miniconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/home/nabeel/miniconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [77]:
train_prob = pd.DataFrame(train_prob)
test_prob = pd.DataFrame(test_prob)

In [78]:
test_probability = test_prob.mean(axis=1).to_numpy()
y_pred = []
for prob in test_probability:
    if prob >=0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
print(classification_report(test["Class"], y_pred))
print(confusion_matrix(test["Class"], y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71079
           1       0.77      0.80      0.78       123

    accuracy                           1.00     71202
   macro avg       0.88      0.90      0.89     71202
weighted avg       1.00      1.00      1.00     71202

[[71049    30]
 [   25    98]]


In [79]:
test_probability.shape

(71202,)

In [80]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_prob, train["Class"])
test_probability = lr.predict(test_prob)
y_pred = []
for prob in test_probability:
    if prob >=0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
print(classification_report(test["Class"], y_pred))
print(confusion_matrix(test["Class"], y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71079
           1       0.78      0.80      0.79       123

    accuracy                           1.00     71202
   macro avg       0.89      0.90      0.89     71202
weighted avg       1.00      1.00      1.00     71202

[[71051    28]
 [   25    98]]
